## Описание проекта
Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. Нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.  
Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

# 1. Подготовка

Загрузим необходимые функции и библиотеки.

In [1]:
import pandas as pd
import re
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import notebook
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Прочтем данные.

In [2]:
comments = pd.read_csv('/datasets/toxic_comments.csv')
comments.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [3]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Проверим комменты на дубликаты и оценку на ошибки.

In [4]:
comments['text'].duplicated().sum()

0

In [5]:
comments['toxic'].unique()

array([0, 1])

Всё ОК.  
Напишем функцию очистки текста от лишних знаков и функцию токенизации и лемматизации. Наш лемматизатор WordNetLemmatizer работает только с отдельными словами, поэтому в аргументах цикл. Для корректной лемматизации английских слов в аргументах также функция, возвращающая кортеж POS-тегов для каждого слова.

In [6]:
w = WordNetLemmatizer()

def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z0-9_-]', ' ', text).split())

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(text):
    str = nltk.word_tokenize(text)
    str_lemma = ' '.join([w.lemmatize(word, get_wordnet_pos(word)) for word in str])
    return str_lemma

Преобразуем наши комментарии и заодно переведем все буквы в нижний регистр (чтобы модель не приняла большие буквы за свойство признака).

In [7]:
for i in notebook.tqdm(range(len(comments.index))):
    comments.loc[i, 'text'] = lemmatize(clear_text(comments.loc[i,'text'])).lower()
    
comments.head(10)

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0
5,congratulations from me a well use the tool we...,0
6,cocksucker before you piss around on my work,1
7,your vandalism to the matt shirvington article...,0
8,sorry if the word nonsense be offensive to you...,0
9,alignment on this subject and which be contrar...,0


Сохраним полученный датасет в файл (слишком долго шла обработка, чтобы терять данные при сбое).

In [8]:
comments.to_csv('comments_preprocessed')

# 2. Обучение

Загрузим предобработанные комменты.

In [9]:
comments_preprocessed = pd.read_csv('comments_preprocessed')

Разобъем выборку на обучающую, валидационную и тестовую.

In [10]:
train, valid_test = train_test_split(
    comments_preprocessed, test_size=0.4, random_state=12345)
valid, test = train_test_split(
    valid_test, test_size=0.5, random_state=12345)

Создадим корпуса и укажем целевые признаки. Также посчитаем TF-IDF для корпусов и сделаем их признаками.

In [11]:
corpus_train = train['text'].values.astype('U')
corpus_valid = valid['text'].values.astype('U')
corpus_test = test['text'].values.astype('U')

target_train = train['toxic']
target_valid = valid['toxic']
target_test = test['toxic']

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
features_train = count_tf_idf.fit_transform(corpus_train)
features_valid = count_tf_idf.transform(corpus_valid)
features_test = count_tf_idf.transform(corpus_test)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Обучим модель решающего дерева с перебором гиперпараметров.

In [12]:
f1_max = 0
crit_list = ['gini', 'entropy']
split_list = ['best', 'random']
for crit in crit_list:
    for split in split_list:
        for depth in range(1, 21, 1):
            model = DecisionTreeClassifier(criterion=crit, splitter=split, random_state=12345, max_depth=depth)
            model.fit(features_train, target_train)
            prediction = model.predict(features_valid)
            accuracy = accuracy_score(target_valid, prediction)
            f1 = f1_score(target_valid, prediction)
            if f1 > f1_max:
                f1_max = f1
                dep = depth
                cr = crit
                spl = split
print('Наибольшее значение F1:', f1_max, 'при:')
print('max_depth =', dep)
print('splitter =', spl)
print('criterion =', cr)

Наибольшее значение F1: 0.6561051004636785 при:
max_depth = 20
splitter = best
criterion = gini


Запустим обучение модели случайного леса с перебором гиперпараметров. Для сглаживания дисбаланса классов укажем class_weight='balanced'.

In [13]:
f1_max = 0
for estim in notebook.tqdm(range(280, 301, 10)):
    for depth in range(24, 30, 1):
        model = RandomForestClassifier(n_estimators=estim, max_depth=depth, random_state=12345, class_weight='balanced')
        model.fit(features_train, target_train)
        prediction = model.predict(features_valid)
        f1 = f1_score(target_valid, prediction)
        if f1 > f1_max:
            f1_max = f1
            est = estim
            dep = depth
print('Наибольшее значение F1:', f1_max, 'при:')
print('n_estimators =', est)
print('max_depth =', dep)


Наибольшее значение F1: 0.4169700807345259 при:
n_estimators = 280
max_depth = 29


Обучим логистическую регрессию.

In [14]:
model = LogisticRegression(random_state=12345, solver='lbfgs', class_weight='balanced')
model.fit(features_train, target_train)
prediction = model.predict(features_valid)
f1_score( target_valid, prediction)

0.7448107448107447

Проверим модель градиетного бустинга Microsoft с перебором гиперпараметров.

In [15]:
f1_max = 0
lr = 0.4
while lr <= 0.6:
    for depth in notebook.tqdm(range(1, 4, 1)):
        model = LGBMClassifier(
                     learning_rate = lr, 
                     max_depth = depth,
                     random_state = 12345, 
                     n_estimators = 500
                                        )

        model.fit(features_train, target_train)
        prediction = model.predict(features_valid)
        f1 = f1_score(target_valid, prediction)
        if f1 > f1_max:
            f1_max = f1
            l_r = lr
            dep = depth
    lr += 0.1
    
print('Наибольшее значение F1:', f1_max, 'при:')
print('learning_rate =', l_r)
print('max_depth =', dep)


Наибольшее значение F1: 0.7605731054721216 при:
learning_rate = 0.4
max_depth = 3


# 3. Выводы

Оценим две лучшие модели еще раз, теперь уже на тестовой выборке. Для оценки адекватности моделей выберем метрику accuracy и создадим модель, где все предсказания - нули.

In [16]:
prediction = pd.Series(0, test.index )
print('Значение accuracy:', accuracy_score(test['toxic'], prediction))

Значение accuracy: 0.8994203352655491


In [17]:
%%time
model = LogisticRegression(
        random_state=12345, 
        solver='lbfgs', 
        class_weight='balanced'
                           )
model.fit(features_train, target_train)

CPU times: user 10.6 s, sys: 15.9 s, total: 26.5 s
Wall time: 26.5 s


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
%%time
prediction = model.predict(features_test)
print('Значение accuracy:', accuracy_score( target_test, prediction))
print('Значение F1:', f1_score( target_test, prediction))

Значение accuracy: 0.940905530314899
Значение F1: 0.7421383647798743
CPU times: user 23.4 ms, sys: 0 ns, total: 23.4 ms
Wall time: 21.4 ms


In [19]:
%%time
model = LGBMClassifier(
        learning_rate = 0.4, 
        max_depth = 3,
        random_state = 12345, 
        n_estimators = 500
                            )
model.fit(features_train, target_train)


CPU times: user 1min 50s, sys: 295 ms, total: 1min 50s
Wall time: 1min 51s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.4, max_depth=3,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
               random_state=12345, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [20]:
%%time
prediction = model.predict(features_test)
print('Значение accuracy:', accuracy_score( target_test, prediction))
print('Значение F1:', f1_score( target_test, prediction))

Значение accuracy: 0.9565721447595175
Значение F1: 0.7580307262569832
CPU times: user 5.06 s, sys: 3.05 ms, total: 5.06 s
Wall time: 5.08 s


Итак, победила модель градиентного бустинга, необходимое значение метрики достигнуто, но логистическая регрессия не дотянула совсем чуть-чуть. В принципе можно рекомендовать обе модели, тем более регрессия обучается и предсказывает намного быстрее. Две другие рассмотренные модели показали слишком низкое значение метрик и не дотягтивают даже до константной модели.